In [1]:
import cv2
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import shutil

from tqdm import tqdm

In [2]:
from tactile_learning.datasets.preprocess import dump_video_to_images, get_closest_id
from tactile_learning.utils.visualization import plot_tactile_sensor

In [3]:
import decimal

# Get the metadata of images and tactile information
def get_desired_indices(root, fps): # frames per second from the video to receive
    image_metadata_path = os.path.join(root, 'cam_0_rgb_video.metadata')
    tactile_info_path = os.path.join(root, 'touch_sensor_values.h5')

    with open(image_metadata_path, 'rb') as f:
        image_metadata = pickle.load(f)
        image_timestamps_array = np.asarray(image_metadata['timestamps'])
        image_timestamps = np.asarray(image_metadata['timestamps']) / 1000.
    with h5py.File(tactile_info_path, 'r') as f:
        tactile_timestamps = f['timestamps'][()]

    image_id, tactile_id = 0, 0
    curr_timestamp = tactile_timestamps[0] # These timestamps are in seconds
    image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

    tactile_indices, image_indices = [], []
    tactile_indices.append(tactile_id)
    image_indices.append(image_id)

    frame_period = 1. / fps
    while(True):
        curr_timestamp += frame_period
        tactile_id = get_closest_id(tactile_id, curr_timestamp, tactile_timestamps)
        image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

        if curr_timestamp > tactile_timestamps[tactile_id] and curr_timestamp > image_timestamps[image_id]:
            break

        tactile_indices.append(tactile_id)
        image_indices.append(image_id)

    assert len(tactile_indices) == len(image_indices)
    return tactile_indices, image_indices


In [4]:

def dump_states(root, tactile_indices, image_indices):
    # Make directory to dump the visualization
    pbar = tqdm(total=len(tactile_indices))

    with h5py.File(os.path.join(root, 'touch_sensor_values.h5'), 'r') as f:
        all_tactile_values = f['sensor_values'][()]

    viz_dir = os.path.join(root, 'visualization')
    os.makedirs(viz_dir, exist_ok=True)
    
    video_path = os.path.join(root, f'cam_0_rgb_video.avi')
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_id = 0
    for i in range(len(tactile_indices)):
        tactile_id, image_id = tactile_indices[i], image_indices[i]
        while frame_id != image_id and success:
            # Find the frame that is equal to image_id
            success, image = vidcap.read()
            frame_id += 1
        dump_demo_state(
            frame_id = i,
            viz_dir = viz_dir,
            tactile_values = all_tactile_values[tactile_id,:,:,:],
            camera_img = image
        )

        pbar.update(1)

    pbar.close()

def dump_demo_state(frame_id, viz_dir, tactile_values, camera_img):
    # tactile_values: (15,16,3)
    fig, axs = plt.subplots(figsize=(20,20), nrows=4, ncols=4)
    for row_id in range(4):
        for column_id in range(4):
            if row_id + column_id > 0: # The top left axis should stay empty
                plot_tactile_sensor(
                    ax = axs[column_id][row_id],
                    sensor_values = tactile_values[row_id*4 + column_id-1],
                    title = f'Sensor {row_id*4+column_id-1}'
                )
    fig.suptitle('Tactile State')
    fig.savefig(os.path.join(viz_dir, 'Tactile State.png'))
    fig.clf()
    plt.close()

    tactile_img = cv2.imread(os.path.join(viz_dir, 'Tactile State.png'))
    height_scale = camera_img.shape[0] / tactile_img.shape[0]
    tactile_img = cv2.resize(
        tactile_img,
        (int(tactile_img.shape[1] * height_scale),
         int(tactile_img.shape[0] * height_scale))
    )
    total_img = cv2.hconcat([camera_img, tactile_img])

    img_name = 'state_{}.png'.format(str(frame_id).zfill(3))
    cv2.imwrite(os.path.join(viz_dir, img_name), total_img)



In [7]:

roots = [
    '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/eval/demonstration_5',
    '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/eval/demonstration_12',
    '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/eval/demonstration_13',
    '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/eval/demonstration_14',
    '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/eval/demonstration_15',
    '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/eval/demonstration_16',
    '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/eval/demonstration_17',
    '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/eval/demonstration_18',
    '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/eval/demonstration_19',
]

retrieval_fps = 2
for root in roots:
    # print('root: {}'.format(root))
    tactile_indices, image_indices = get_desired_indices(root=root, fps=retrieval_fps)
    print('tactile_indices: {}, image_indices: {}'.format(tactile_indices, image_indices))
    dump_states(root, tactile_indices, image_indices)


tactile_indices: [0, 51, 100, 150, 199, 248, 297, 347, 395, 445, 499, 544, 592, 642, 691, 740, 790, 839, 887, 937, 985, 1035, 1082, 1132], image_indices: [74, 88, 103, 118, 133, 148, 163, 178, 193, 208, 223, 238, 252, 267, 282, 297, 311, 326, 341, 356, 371, 386, 401, 416]


100%|█████████████████████| 24/24 [00:27<00:00,  1.16s/it]


tactile_indices: [0, 51, 104, 150, 198, 249, 297, 347, 395, 444, 493, 543, 592, 642, 691, 739, 788, 836, 886, 935, 985, 1034, 1083, 1133, 1181, 1230, 1279, 1328, 1378, 1426, 1476, 1525, 1575, 1624, 1674, 1723, 1773, 1820, 1871, 1919, 1969, 2018, 2066, 2116, 2165, 2218, 2263, 2314, 2361, 2411, 2459, 2508], image_indices: [69, 84, 99, 113, 128, 143, 158, 173, 188, 203, 217, 232, 247, 262, 277, 292, 307, 320, 335, 350, 365, 380, 395, 409, 424, 439, 453, 468, 483, 498, 513, 528, 543, 558, 573, 588, 603, 618, 633, 648, 663, 678, 693, 708, 723, 738, 753, 768, 783, 798, 813, 828]


100%|█████████████████████| 52/52 [00:59<00:00,  1.14s/it]


tactile_indices: [0, 50, 99, 149, 197, 246, 294, 345, 394, 443, 492, 540, 590, 639, 691, 739, 787, 837, 887, 936, 984, 1034, 1082, 1132, 1182, 1231, 1280, 1329, 1378, 1427, 1478, 1525, 1576], image_indices: [68, 83, 98, 113, 128, 143, 158, 173, 188, 203, 218, 233, 248, 263, 278, 293, 308, 323, 338, 353, 368, 382, 396, 411, 426, 441, 456, 471, 486, 501, 515, 530, 545]


...

100%|█████████████████████| 55/55 [01:02<00:00,  1.13s/it]


tactile_indices: [0, 52, 100, 150, 200, 249, 299, 348, 398, 446, 496, 544, 595, 643, 693, 742, 792, 840, 890, 938, 989, 1037, 1087, 1136, 1189, 1234, 1285, 1332, 1383, 1429, 1481, 1529, 1579], image_indices: [70, 85, 100, 115, 130, 145, 159, 174, 189, 204, 219, 233, 248, 262, 277, 292, 307, 322, 337, 352, 366, 381, 396, 411, 426, 441, 456, 470, 484, 499, 514, 529, 544]


100%|█████████████████████| 33/33 [00:37<00:00,  1.12s/it]


tactile_indices: [0, 51, 100, 152, 198, 248, 296, 346, 396, 445, 494, 544, 592, 642, 690, 740, 788, 841, 887, 937, 985, 1036, 1083, 1133, 1182, 1232], image_indices: [71, 86, 101, 116, 131, 146, 161, 176, 191, 206, 221, 236, 251, 265, 280, 295, 310, 325, 339, 354, 368, 383, 398, 413, 428, 443]


100%|█████████████████████| 26/26 [00:28<00:00,  1.08s/it]


In [8]:
# Turn the images to a video and delete the directory
video_fps = 10
for root in roots:
    print('dumping video in root: {}'.format(root))
    video_path = os.path.join(root, 'visualization.mp4')
    if os.path.exists(video_path):
        os.remove(video_path)
    viz_dir = os.path.join(root, 'visualization')
    os.system('ffmpeg -r {} -i {}/%*.png -vf scale=2000x720,setsar=1:1 {}'.format(
        video_fps, # fps
        viz_dir,
        video_path
    ))


dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/eval/demonstration_5


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-18)
  configuration: --prefix=/home/irmak/miniconda3/envs/tactile_learning --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/eval/demonstration_12


[libx264 @ 0x555dbe280f40] using SAR=1/1
[libx264 @ 0x555dbe280f40] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x555dbe280f40] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x555dbe280f40] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_l

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/eval/demonstration_13


...

[libx264 @ 0x558c0b516580] using SAR=1/1
[libx264 @ 0x558c0b516580] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x558c0b516580] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x558c0b516580] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_l

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/eval/demonstration_18


[libx264 @ 0x5645ac1ebac0] using SAR=1/1
[libx264 @ 0x5645ac1ebac0] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x5645ac1ebac0] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x5645ac1ebac0] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_l

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_location_changing/eval/demonstration_19


[libx264 @ 0x562b21bb48c0] using SAR=1/1
[libx264 @ 0x562b21bb48c0] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x562b21bb48c0] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x562b21bb48c0] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/box_handle_lifting/box_l

In [9]:
# Remove the img directory

for root in roots:
    viz_dir = os.path.join(root, 'visualization')
    shutil.rmtree(viz_dir)
